In [1]:
from tqdm import tqdm, trange
import numpy as np
import copy

data_dir = "/home/jack/Desktop/NN/clean/datasets/yelp"

In [2]:
def process_file(input_file, output_file):
    """
    The function process the data files for Delete & Generate and convert
    it for the Delete, Retrieve and Generate training by separating the content
    and attributes. It includes all the attribure words.
    
    Input_file: string : Path of the input file
    Output_file: string : Path of the output file 
    """
    
    with open(input_file) as fp:
        data = fp.read().splitlines()
    with open (output_file,"w") as out_fp:
        for x in tqdm(data):
            temp = x.split("<START>")
            con = temp[0].replace("<POS>","").replace("<NEG>","").replace("<CON_START>","")
            sen = temp[1].replace("<END>","")
            lt1 = con.split()
            lt2 = sen.split()
            att_tokens = [z for z in lt2 if z not in lt1]
            max_atts = 0
            if len(att_tokens) > max_atts:
                max_atts = len(att_tokens)
            att_words = " ".join(att_tokens)
            out_str = "<ATTR_WORDS> " + att_words + " <CON_START> " + con.strip() + " <START> " + sen.strip() + " <END>" + "\n"
            out_fp.write(out_str)
    

In [221]:
def process_file_v1(input_file, output_file, test = False):
    """
    The function process the data files for Delete & Generate and convert
    it for the Delete, Retrieve and Generate training by separating the content
    and attributes. 
    It randomly picks 70% of the attributes only to make the generation
    more realistic instead of just filling the blanks, which helps while generating
    sentences for test cases.
    
    Input_file: string : Path of the input file
    Output_file: string : Path of the output file 
    """
    with open(input_file) as fp:
        data = fp.read().splitlines()
    with open (output_file,"w") as out_fp:
        for x in tqdm(data):
            temp = x.split("<START>")
            con = temp[0].replace("<POS>","").replace("<NEG>","").replace("<CON_START>","")
            sen = temp[1].replace("<END>","")
            #print("con", con)
            #print("sen", sen)
            lt1 = con.split()
            org_lt1 = copy.deepcopy(lt1)
            lt2 = sen.split()
           
            
            att_words = [z for z in lt2 if z not in lt1]
            att_words = list(reversed(sorted(att_words, key=len)))
            index_att = []

            #print(len(att_words),att_words)
            #print(len(lt1),lt1)
            #print(len(lt2),lt2)
            
            # Don't put special char in attribute words. Reduce att -> reduce confusion
            my_att = []

            special_characters =  "!@#$%^&*()-+?_=<>/\'\'"
            for index, word  in enumerate(lt2) :
                if word in special_characters:
                    continue
                if word in lt1:
                    continue
                if any(c in special_characters for c in word) and len(word) < 3:
                    continue
                if "-" in word:
                    splitted_words = word.split("-")
                    if splitted_words[0] in lt1 and splitted_words[1] in lt1:
                        continue
                if any(c in special_characters for c in word) and len(word) < 4:
                    splitted_words = word.split("\'")
                    if len(splitted_words)>0 and splitted_words[0] in lt1 and splitted_words[1] in lt1:
                        continue
                # Attribute found
                my_att.append(word)
                # Remember index
                index_att.append(index)
            

            att_words = my_att

            #print("lenlt1", len(lt1), len(att_words))
            #print("lenlt2", len(lt2))
            #if len(att_words) != len(my_att):
            #print(len(att_words) != len(my_att))
            #print(len(att_words),att_words)
            #print(len(my_att),my_att)

            


                
                #if lt1[replace_index - 1] == lt2[replace_index - 1] and lt1[replace_index + 1] == lt2[replace_index + 1]:
                    #print("AAAAAAAAA") 
                    #print(lt2[replace_index]) 
                    #print(lt1[replace_index]) 
                    #lt1.insert(replace_index, "<REPLACE>")
                #print(replace_index)
                

            set_replace_tokens(lt1, lt2, my_att, index_att)

            if len(att_words) > 2:
                indx = np.array(list(range(len(att_words))))
                # Pref delete short words
                att_words = " ".join([att_words[indx[k]] for k in range(int(0.7 * len(att_words)))])
                #np.random.shuffle(indx)
            else: # If attributes less than 2 then keep all the attributes
                att_words = " ".join(att_words)
            #print("att_words", att_words)
            #print("sen: ", sen)
            #print("con: ", con)


            # TODO HERE MODIFIY CON with <REPLACE> to show position of word
            # Also put Token <REPLACE> in reference file 
            if(test):
                out_str = "<ATTR_WORDS> " + att_words + " <CON_START> " +  " ".join(lt1).strip() + " <START> " + "\n"
            else:
                out_str = "<ATTR_WORDS> " + att_words + " <CON_START> " +  " ".join(lt1).strip() + " <START> " + sen.strip() + " <END>"  + "\n"
            out_fp.write(out_str)

def set_replace_tokens(content_list, full_list, att_list, index_att_list, second_try = False):
    #print(content_list, full_list, att_list, index_att_list)
    street_index = 0
    last_replace_index = None
    insert_index = None
    for index, replace_index in enumerate(index_att_list):
        # Catch street 
        if last_replace_index == replace_index - 1:
            content_list.insert(replace_index, "<REPLACE>")
            last_replace_index = replace_index
            continue

        last_replace_index = replace_index
        # Catch first street if starts with 0
        if replace_index == street_index:
            street_index += 1
            content_list.insert(replace_index, "<REPLACE>")
            continue

        
            
        # TODO DEbug that after remove
        #out_of_index = False
        #if replace_index+1 >= len(content_list):
        #    out_of_index = True
        #    continue
        left_matches = []
        right_matches = []
        left_word_query = None
        right_word_query = None
        
        if replace_index-1 >= 0:
            left_word_query = full_list[replace_index-1]
            left_matches = [(index,x) for index,x in enumerate(content_list) if x == left_word_query]

        if replace_index+1 < len(full_list):
            right_word_query = full_list[replace_index+1]
            right_matches = [(index,x) for index,x in enumerate(content_list) if x == right_word_query]

        if len(left_matches) == 1:
            insert_index = left_matches[0][0]+1
            content_list.insert(insert_index, "<REPLACE>")

            #for x in range(get_street_length(index_att_list, replace_index)):
               # replace_index += 1
               # insert_index += 1
               # content_list.insert(insert_index, "<REPLACE>")
            continue
        
        if len(right_matches) == 1:
            insert_index = right_matches[0][0]
            content_list.insert(insert_index, "<REPLACE>")            
            continue
            #print("RIGHT")


        left_concat_matches = []
        right_concat_matches = []
        if left_word_query != None:
            left_concat_matches = [(index,x) for index,x in enumerate(content_list) if index-1 > -1 and content_list[index-1]+content_list[index] == left_word_query]
        if right_word_query != None:
            right_concat_matches = [(index,x) for index,x in enumerate(content_list) if index+1 < len(content_list) and content_list[index]+content_list[index+1] == right_word_query]
        
        if len(left_concat_matches) == 1: 
            insert_index = left_concat_matches[0][0]+1
            content_list.insert(insert_index, "<REPLACE>")
            continue

        if len(right_concat_matches) == 1: 
            insert_index = right_concat_matches[0][0]
            content_list.insert(insert_index, "<REPLACE>")
            continue


        left_shifted_match = []
        right_shifted_matches = []
        if replace_index-2 >= 0:
            left_word_query_shift = full_list[replace_index-2]
            left_shifted_match = [(index,x) for index,x in enumerate(content_list) if index-1 > -1 and content_list[index-1] == left_word_query_shift]
        if replace_index+2 < len(full_list):
            right_word_query_shift = full_list[replace_index+2]
            right_shifted_matches = [(index,x) for index,x in enumerate(content_list) if index+1 < len(content_list) and content_list[index+1] == right_word_query_shift]

        if len(left_shifted_match) == 1: 
            insert_index = left_shifted_match[0][0]+1
            content_list.insert(insert_index, "<REPLACE>")
            continue

        if len(right_shifted_matches) == 1: 
            insert_index = right_shifted_matches[0][0]
            content_list.insert(insert_index, "<REPLACE>")
            continue

        # insert street check routine

        if len(left_matches) == 0 and len(right_matches) == 0:
            print("NONE", index)
            print(att_list)
            print(index_att_list)
            print(content_list)
            print(full_list)
            #if second_try == False:
                #set_replace_tokens(content_list, full_list, list(reversed(att_list)), list(reversed(index_att_list)), True)

def next_is_street(index_att_list, current_index):
    #print(len(index_att_list) > current_index and  index_att_list[current_index] == index_att_list[current_index] + 1)

    if len(index_att_list) > current_index+1 and  index_att_list[current_index] + 1 == index_att_list[current_index+1]:
        return True
    else:
        return False

def get_street_length(index_att_list, current_index):
    counter = 0
    while len(index_att_list) > current_index+1 and  index_att_list[current_index] + 1 == index_att_list[current_index+1]:
        counter += 1
        current_index += 1
    return counter



In [184]:
def get_street_length(index_att_list, current_index):
    counter = 0
    #print(len(index_att_list) > current_index+1)
    #print(index_att_list[current_index] + 1 == index_att_list[current_index+1])
    while len(index_att_list) > current_index+1 and  index_att_list[current_index] + 1 == index_att_list[current_index+1]:
        counter += 1
        current_index += 1:
            left_word_query = full_list[replace_index-1]
            left_matches = [(index,x) for index,x in enumerate(content_list) if x == left_word_query]

for x in  range( get_street_length([0,1,2,5],2)):
    aa = [0,1,2,5]
    #print("aa",aa[4])
    print(x)

In [209]:
# Test changes here first
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test.txt")
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_1.txt")
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_0.txt")

100%|██████████| 500/500 [00:00<00:00, 44968.52it/s]


In [44]:
'''Merge files'''
# Create sentiment_train file before run!
# Create sentiment_test file before run!
# Create sentiment_dev file before run!

filenames = [data_dir+"/processed_files_with_bert_with_best_head/sentiment_train_1.txt", data_dir+"/processed_files_with_bert_with_best_head/sentiment_train_0.txt"]
with open(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train.txt", 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)


filenames = [data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_1.txt", data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_0.txt"]
with open(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test.txt", 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

filenames = [data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev_1.txt", data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev_0.txt"]
with open(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev.txt", 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [14]:

process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train_all_attrs.txt")
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train_1_all_attrs.txt")
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train_0_all_attrs.txt")

100%|██████████| 177218/177218 [00:00<00:00, 316493.76it/s]


In [12]:
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_all_attrs.txt")
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_1_all_attrs.txt")
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_0_all_attrs.txt")

100%|██████████| 500/500 [00:00<00:00, 269660.79it/s]


In [ ]:
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_dev_all_attrs.txt")
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_dev_1_all_attrs.txt")
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_dev_0_all_attrs.txt")

In [223]:
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train.txt")
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train_1.txt")
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train_0.txt")

  3%|▎         | 12511/443259 [00:00<00:06, 62731.38it/s]

NONE 0
['well', 'worth']
[9, 10]
['but', 'even', 'if', 'you', 'have', 'to', 'wait', 'it', "'", 'it', '.']
['but', 'even', 'if', 'you', 'have', 'to', 'wait', 'it', "'s", 'well', 'worth', 'it', '.']


 10%|▉         | 43785/443259 [00:00<00:06, 61969.08it/s]

NONE 1
['yes', 'well', 'worth']
[0, 8, 9]
['<REPLACE>', ',', 'it', "'", 's', 'pricey', 'but', 'it', "'", 's', 'it', '!']
['yes', ',', 'it', "'s", 'pricey', 'but', 'it', "'s", 'well', 'worth', 'it', '!']


 14%|█▍        | 62692/443259 [00:01<00:06, 62600.83it/s]

...

NONE 0
['_num_', 'bucks', 'lunch', '_num_']
[1, 2, 4, 6]
['_', 'num', '_', 'for', 'for', '_', 'num', '_', '.']
['$', '_num_', 'bucks', 'for', 'lunch', 'for', '_num_', '.']
NONE 1
['tea', 'hunger', 'cravings']
[1, 4, 5]
['for', '<REPLACE>', ',', 'late', '-', 'night', ',', 'dinner', ',', 'lunch', ',', 'dessert', '.']
['for', 'tea', ',', 'late-night', 'hunger', 'cravings', ',', 'dinner', ',', 'lunch', ',', 'dessert', '.']


 94%|█████████▍| 166854/177218 [00:03<00:00, 52768.83it/s]

NONE 1
['did', 'taste', 'horrible', '...']
[1, 3, 4, 5]
['they', '<REPLACE>', 'n', "'", 't', '.', '.', '.', 'but', 'you', 'could', 'n', "'", 't', 'dip', 'them', 'into', 'anything', '.']
['they', 'did', "n't", 'taste', 'horrible', '...', 'but', 'you', 'could', "n't", 'dip', 'them', 'into', 'anything', '.']
NONE 1
['do', 'waste', '_num_', 'months']
[0, 2, 3, 4]
['<REPLACE>', 'n', "'", 't', '_', 'num', '_', 'waiting', 'on', 'broken', 'promises', 'and', 'hard', 'luck', 'stories', '.']
['do', "n't", 'waste', '_num_', 'months', 'waiting', 'on', 'broken', 'promises', 'and', 'hard', 'luck', 'stories', '.']
NONE 0
['_num_', 'gold', 'club']
[1, 2, 3]
['_', 'num', '_', 'member', 'with', 'a', 'reservation', '?']
['#', '_num_', 'gold', 'club', 'member', 'with', 'a', 'reservation', '?']


100%|██████████| 177218/177218 [00:03<00:00, 51339.26it/s]


NONE 0
['_num_', 'n.', '_num_', 'rd']
[1, 2, 3, 4]
['_', 'num', '_', '.', '_', 'num', '_', '.']
['@', '_num_', 'n.', '_num_', 'rd', '.']
NONE 3
['discovered', 'pads', '_num_', 'x', '_num_']
[4, 6, 8, 10, 11]
['that', "'", 's', 'where', 'i', '<REPLACE>', 'the', '<REPLACE>', 'were', '<REPLACE>', '_', 'num', '_', "'", "'", '_', 'num', '_', "'", "'", '.']
['that', "'s", 'where', 'i', 'discovered', 'the', 'pads', 'were', '_num_', "''", 'x', '_num_', "''", '.']


In [158]:
# Test changes here first
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test.txt")
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_1.txt")
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_0.txt")

100%|██████████| 500/500 [00:00<00:00, 66485.50it/s]


In [217]:
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_dev.txt", test=True)
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_dev_1.txt", test=True)
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_dev_0.txt", test=True)

100%|██████████| 4000/4000 [00:00<00:00, 44698.12it/s]


NONE 0
['_num_', 'wi-fi', 'charge']
[1, 2, 3]
['_', 'num', '_', '-', 'fi', '.']
['$', '_num_', 'wi-fi', 'charge', '.']


100%|██████████| 2000/2000 [00:00<00:00, 43963.84it/s]


NONE 0
['_num_', 'wi-fi', 'charge']
[1, 2, 3]
['_', 'num', '_', '-', 'fi', '.']
['$', '_num_', 'wi-fi', 'charge', '.']


100%|██████████| 2000/2000 [00:00<00:00, 45756.63it/s]
